# Developing New method for Using Kamodo with GEODYN

*This new method should run the orbits of GEODYN through Kamodo in a faster way that doesn't sacrifice accuracy of the interpolation in Kamodo and doesn't require a trimming of other Kamodo functionality to accomodate large overhead.*

1. Pre-run initialization of the orbit.
   - "initialize" a run of GEODYN with MSIS2 that gives us a "best guess" for the orbit of the satellite.
2. Construct box of estimated orbits with variation
   - Before running GEODYN with TIEGCM, we will take the "best guess" orbit and perturbate the input coordinates outward for each timestep that GEODYN asks for.  This will construct a box of best guesses around our satellite that can then be plugged into Kamodo.  
   - From here we just have a meshed index of possible density values from Kamodo for each timepoint and within some standard deviation of possible values along the orbit of the s/c.
3. Get density in GEODYN
   - now instead of calling the Kamodo model at each time step which has tons of overhead, we can call a relatively simple interpolation scheme in fortran at each timestep that will return the density value.  The orbit grid will be saved in a file that will be indexed by time according to what the MSIS model returned.
   - We will need to build in functionality such that if GEODYN requests a value outside of the box we requested, then it stops the program and restarts the process with a slightly larger grid around the orbit.
   
   


## Function for trilinear interpolation

In [1]:

def interpolate_point_in_cube(lon, lat, alt, uncertain_grid, timestep, delta_deg, delta_m):
    '''
    This function performs a manual calculation for trilinear interoplation.
    Interpolates values on a cube to a desired coordinate.  
    Currently, it is set up for Lon, Lat, Alt.
    '''    
    x  = lon
    x0 = x - delta_deg
    x1 = x + delta_deg

    y  = lat
    y0 = y - delta_deg
    y1 = y + delta_deg

    z  = alt
    z0 = z - delta_m
    z1 = z + delta_m

    C000 = np.array(uncertain_grid[timestep]['values'][5])  # bottom, front, left
    C100 = np.array(uncertain_grid[timestep]['values'][7])  # bottom, front, right
    C001 = np.array(uncertain_grid[timestep]['values'][1])  # top,    front, left
    C101 = np.array(uncertain_grid[timestep]['values'][3])  # top,    front, right
    C010 = np.array(uncertain_grid[timestep]['values'][6])  # bottom, back,  left
    C110 = np.array(uncertain_grid[timestep]['values'][8])  # bottom, back,  right
    C011 = np.array(uncertain_grid[timestep]['values'][2])  # top,    back,  Left
    C111 = np.array(uncertain_grid[timestep]['values'][4])  # top,    back,  right


    ##===================================================================
    ##        Manually INTERPOLATE-- Trilinear Interpolation Calculation
    ##===================================================================

    ### On a periodic and cubic lattice, let xd, yd, zd be the differences 
    ### between each of x, y, z and the smaller coordinate related.
    #
    ###  x0 indicates the lattice point below x 
    ###  x1 indicates the lattice point above x
    xd =  (x-x0)/(x1-x0)
    yd =  (y-y0)/(y1-y0)
    zd =  (z-z0)/(z1-z0)
    # print(xd,yd,zd)

    ##### First we interpolate along x:   (push one of the X faces of cube towards the opposing face)
    ###   C000 uis the function value (x0, y0, z0)
    C00 = C000*(1-xd) + (C100*xd)
    C01 = C001*(1-xd) + (C101*xd)
    C10 = C010*(1-xd) + (C110*xd)
    C11 = C011*(1-xd) + (C111*xd)

    #### Next we interpolate along y:  (pushing the values that are now in middle of cube towards the center Y)
    C0 = C00*(1-yd) + C10*yd
    C1 = C01*(1-yd) + C11*yd

    #### Finally we interpolate along Z   (walk through the line that remains)
    C = C0*(1-zd) + C1*zd
    
    return(C)

## Do a Pre-run with MSIS2

In [2]:
# %load_ext autoreload
# %autoreload 2
# import copy
# import time
# import sys  


# sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
# from PYGEODYN import Pygeodyn


# ### Identify which arcs you want to run:
# arcs_files = [ 
#             '031109',  # 2
#              ]

# #------ A dictionary containing the run parameters ------  
# run_params = {} 
# run_params['arc']              =   arcs_files
# run_params['satellite']        =  'starlette'  
# run_params['den_model']        =  'msis2'  
# run_params['SpecialRun_name']  =  'SLR_kamodo_interface'  
# run_params['verbose']          =  False

# %load_ext autoreload
# %autoreload 2

# import pickle 
# sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
# from PYGEODYN import Pygeodyn

# for imodel,val_model in enumerate( ['msis2']):
#     run_params1 = copy.deepcopy(run_params)
#     run_params1['den_model']  =  val_model  
#     run_params1['action']           =  'run'

#     ### Load the data into an object
#     Obj_Geodyn = Pygeodyn(run_params1)
#     Obj_Geodyn.RUN_GEODYN()
#     del Obj_Geodyn
    

## Read Density Values from Pre-Run

In [3]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import numpy as np


#### Need to bunzip the file we want to play with.

!bunzip2 /data/data_geodyn/results/st/msis2/msis2_acceloffSLR_kamodo_interface/DENSITY/st031109_2wk.goco05s.bz2

bunzip2: Can't open input file /data/data_geodyn/results/st/msis2/msis2_acceloffSLR_kamodo_interface/DENSITY/st031109_2wk.goco05s.bz2: No such file or directory.


In [4]:

DEN_csv = pd.read_csv('/data/data_geodyn/results/st/msis2/msis2_acceloffSLR_kamodo_interface/DENSITY/st031109_2wk.goco05s', 
                    skiprows = 1, 
                    dtype=object,
                    names = ['Elapsed Secs',
                            'YYMMDD',
                            'HHMMSS',
                            'Lat',
                            'Lon',
                            'Height (meters)',
                            'rho (kg/m**3)',
                            'drhodz (kg/m**3/m)',
                             'flux_daily',
                            'flux_avg',
                            'Kp',
                          ],
                    sep = '\s+',
                    )
sat_time1 = list(DEN_csv['YYMMDD'])  #"031115" #  
sat_time2 = list(DEN_csv['HHMMSS'])  #"120000" #1068897600        
sattime   =    [x+y   for x,y   in zip(sat_time1, sat_time2)]

sattime   =    [datetime.strptime(x, '%y%m%d%H%M%S')   for x   in sattime ]
sattime   =    [datetime.timestamp(x)   for x   in sattime ]
DEN_csv['sattime_utctimestamp'] = sattime
DEN_csv['Height_kilometers'] = DEN_csv['Height (meters)'].astype(float)*1e-3
DEN_csv['Lon'] = DEN_csv['Lon'].astype(float)
DEN_csv['Lat'] = DEN_csv['Lat'].astype(float)

fix_D_decimal_to_E = []
fix_D_decimal_to_E2 = []

for i,val in enumerate(DEN_csv['rho (kg/m**3)']):
    val2 = DEN_csv['drhodz (kg/m**3/m)'][i]

    list_val = list(val)
    list_val2 = list(val2)

    indx = list(val).index('D')
    indx2 = list(val2).index('D')

    list_val[indx] = 'E'
    list_val2[indx2] = 'E'

    list_val = "".join(list_val)
    list_val2 = "".join(list_val2)

    #### If you get an error here, it is likely:
    ####
    ###

    val_float = np.float(list_val)
    val_float2 = np.float(list_val2)

    fix_D_decimal_to_E.append(val_float)
    fix_D_decimal_to_E2.append(val_float2)

DEN_csv['rho (kg/m**3)'] = fix_D_decimal_to_E
DEN_csv['drhodz (kg/m**3/m)'] = fix_D_decimal_to_E2
timeHHMMSS = [] 
for i,val in enumerate(DEN_csv['HHMMSS'].values.astype(int)):
    # print(len(str(val)))
    if len(str(val)) == 1:
        timehhmmss_val = '00000'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 2:
        timehhmmss_val = '0000'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 3:
        timehhmmss_val = '000'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 4:
        timehhmmss_val = '00'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 5:
        timehhmmss_val = '0'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    else:
        timeHHMMSS.append(str(val))
DEN_csv['timeHHMMSS'] = timeHHMMSS
YR = int(3)

YYMMDD_list = DEN_csv['YYMMDD'].astype(int).astype(str)
timeHHMMSS_list = DEN_csv['timeHHMMSS'].astype(str)

if YR < 10:
    year    = ['200' + x[:1]  for x in YYMMDD_list]
    month   = [        x[1:3] for x in YYMMDD_list]
    day     = [        x[3:]  for x in YYMMDD_list]
    hours   = [        x[:2]  for x in timeHHMMSS_list]
    minutes = [        x[2:4] for x in timeHHMMSS_list]
    secs    = [        x[4:]  for x in timeHHMMSS_list]
else:
    year    = ['20' + x[:2]  for x in YYMMDD_list]
    month   = [       x[2:4] for x in YYMMDD_list]
    day     = [       x[4:]  for x in YYMMDD_list]
    hours   = [       x[:2]  for x in timeHHMMSS_list]
    minutes = [       x[2:4] for x in timeHHMMSS_list]
    secs    = [       x[4:]  for x in timeHHMMSS_list]
#--------------------------------------------------------
DEN_csv['year']  = year
DEN_csv['month'] = month
DEN_csv['day']   = day
DEN_csv['hours']  = hours
DEN_csv['minutes'] = minutes
DEN_csv['secs']  = secs
#--------------------------------------------------------
year= list(map(int, DEN_csv['year'].values))
month= list(map(int, DEN_csv['month'].values))
day= list(map(int, DEN_csv['day'].values))
hour= list(map(int, DEN_csv['hours'].values))
minute = list(map(int, DEN_csv['minutes'].values))
second = list(map(int, DEN_csv['secs'].values))

DATE = list(map(datetime, year,month, day, hour,minute,second ))

#self.DEN_df['Date']  = DATE
DEN_csv.insert(0, 'Date', DATE)

del DEN_csv['timeHHMMSS']
del DEN_csv['year']
del DEN_csv['month']
del DEN_csv['day']
del DEN_csv['hours']
del DEN_csv['minutes']
del DEN_csv['secs']


DEN_csv

<ipython-input-4-8b7adbf804b3>:51: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  val_float = np.float(list_val)
<ipython-input-4-8b7adbf804b3>:52: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  val_float2 = np.float(list_val2)


,Date,Elapsed Secs,YYMMDD,HHMMSS,Lat,Lon,Height (meters),rho (kg/m**3),drhodz (kg/m**3/m),flux_daily,flux_avg,Kp,sattime_utctimestamp,Height_kilometers
0,2003-11-09 00:07:04,360.0,031109,000704,-25.1728,217.8688,1113435.865,1.914916e-15,-7.201987e-21,91.47,133.12,4.67,1.068336e+09,1113.435865
1,2003-11-09 00:08:04,420.0,031109,000804,-22.8145,220.1761,1113033.012,1.965050e-15,-7.402988e-21,91.48,133.12,4.67,1.068336e+09,1113.033012
2,2003-11-09 00:09:04,480.0,031109,000904,-20.4158,222.3964,1112128.477,2.018587e-15,-7.624048e-21,91.48,133.12,4.67,1.068337e+09,1112.128477
3,2003-11-09 00:10:04,540.0,031109,001004,-17.9820,224.5414,1110730.463,2.074163e-15,-7.859084e-21,91.48,133.12,4.67,1.068337e+09,1110.730463
4,2003-11-09 00:11:04,600.0,031109,001104,-15.5180,226.6222,1108848.344,2.130286e-15,-8.101387e-21,91.48,133.12,4.67,1.068337e+09,1108.848344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20145,2003-11-09 00:05:04,240.0,031109,000504,-29.7438,212.9431,1112710.164,1.828878e-15,-6.878285e-21,91.47,133.12,4.67,1.068336e+09,1112.710164
20146,2003-11-09 00:05:04,240.0,031109,000504,-29.7438,212.9431,1112710.164,1.828878e-15,-6.878285e-21,91.47,133.12,4.67,1.068336e+09,1112.710164
20147,2003-11-09 00:05:04,240.0,031109,000504,-29.7438,212.9431,1112710.164,1.828878e-15,-6.878285e-21,91.47,133.12,4.67,1.068336e+09,1112.710164
20148,2003-11-09 00:05:04,240.0,031109,000504,-29.7438,212.9431,1112710.164,1.828878e-15,-6.878285e-21,91.47,133.12,4.67,1.068336e+09,1112.710164


## Make a cube of uncertainty around the init_orbit

**Considerations**
 - its possible that as long as our resolution is on par with MSIS we aren't introducing additional error. 
 - Gauss legendre sampling scheme
 - Perhaps can access geodyn's sampling method in the early iterations.
 - nearest neighbor interpolation within Fortran for optimized speed. Weighted average for distance between each point
 - how much memory to story the data-table in memory.
  - we will make MSISsubset  using msis


**Desired Features in the Data Structure**
 - Must be able to interpolate to a desired location
 - Should be able to expand the bounds of uncertainty at any given timestep along the orbit
    - prefereably without having to redo the already constructed portions
 - Each timestep should contain information on the `Lon`, `Lat`, `Alt` as well as the `RHO` and `DRHODZ` at each location
 - Include GEODYN run info (Satellite, Arc#, Arc Dates, density model used to initialize, etc.)


Parent: timestep  
    - each parent has square that contains uncertainty around the orbit 
    - lon lat, alt




In [5]:
TEST_NUMS = 5

**START WITH A SIMPLE BOX**
Just draw a cube around the coordinate

In [6]:
import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SingleSatelliteFlythrough import SingleModelFlythrough
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SatelliteFlythrough import ModelFlythrough


uncertain_grid = {}

for i,val in enumerate(DEN_csv['Elapsed Secs'][:TEST_NUMS]):
    ## Make each timestep a dictionary to hold our coordinates
    date_str = DEN_csv['YYMMDD'][i] + DEN_csv['HHMMSS'][i]
    date_index = datetime.strptime(date_str, '%y%m%d%H%M%S')

    
    unix_time  = DEN_csv['sattime_utctimestamp'][i]
    print('****',date_index, '****' )

    
    uncertain_grid[date_index] = {}
    uncertain_grid[date_index]['coords'] = []
    uncertain_grid[date_index]['values'] = []
    
    
    ### Get the coordinates along the orbit:
    lon = float(DEN_csv['Lon'][i])
    lat = float(DEN_csv['Lat'][i])
    alt = float(DEN_csv['Height_kilometers'][i])
    center_coord = [lon, lat, alt]
    
    
    ### Find the coordinates of the cube surround the orbit point:
    delta_deg = 2    # degrees
    delta_m = 1000.*1e-3 # meters to kilometers
    A = [lon + delta_deg, lat+delta_deg, alt+delta_m]  # top,    front, left
    B = [lon + delta_deg, lat-delta_deg, alt+delta_m]  # top,    back,  Left
    C = [lon - delta_deg, lat+delta_deg, alt+delta_m]  # top,    front, right
    D = [lon - delta_deg, lat-delta_deg, alt+delta_m]  # top,    back,  right
    E = [lon + delta_deg, lat+delta_deg, alt-delta_m]  # bottom, front, left
    F = [lon + delta_deg, lat-delta_deg, alt-delta_m]  # bottom, back,  left
    G = [lon - delta_deg, lat+delta_deg, alt-delta_m]  # bottom, front, right
    H = [lon - delta_deg, lat-delta_deg, alt-delta_m]  # bottom, back,  right
    
    
    ### Store the cube's coordinates in the dictionary index
    uncertain_grid[date_index]['coords'].append(center_coord)
    uncertain_grid[date_index]['coords'].append(A)
    uncertain_grid[date_index]['coords'].append(B)
    uncertain_grid[date_index]['coords'].append(C)
    uncertain_grid[date_index]['coords'].append(D)
    uncertain_grid[date_index]['coords'].append(E)
    uncertain_grid[date_index]['coords'].append(F)
    uncertain_grid[date_index]['coords'].append(G)
    uncertain_grid[date_index]['coords'].append(H)
    
    
    #### Import Coordinates to Kamodo
    ##
    #### Kamodo static inputs:
    model          = 'TIEGCM'
    file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
    variable_list  = ['rho','psi_O2', 'psi_O', 'psi_N2', 'psi_He', 'T_n']
    coord_type     = 'SPH'
    coord_grid     = 'sph'
    high_res       = 1.
    verbose        = False  
    csv_output     = '' 
    plot_output    = ''
    
    
    #### Extract the coordinates from each list to plug into Kamodo with vectorization
    lons_in = [item[0] for item in uncertain_grid[date_index]['coords']]
    lats_in = [item[1] for item in uncertain_grid[date_index]['coords']]
    alts_in = [item[2] for item in uncertain_grid[date_index]['coords']]
    ## Gather inputs
    sat_time       = unix_time*np.ones(np.size(alts_in))
    c1             = lons_in
    c2             = lats_in
    c3             = alts_in
    ## Plug vectorized coordinates into Kamodo
    results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
                        coord_type, coord_grid, high_res=20., verbose=False, 
                        csv_output='', plot_output='')

    
    #### Save the values from Kamodo to the timestep:
    uncertain_grid[date_index]['values'] = [[rho] for rho in results['rho']]    
    


import time: 1.3721626379992813
**** 2003-11-09 00:07:04 ****
**** 2003-11-09 00:08:04 ****
**** 2003-11-09 00:09:04 ****
**** 2003-11-09 00:10:04 ****
**** 2003-11-09 00:11:04 ****


In [7]:
orbit_w_uncertainty_cube = uncertain_grid

In [8]:
# for val in orbit_w_uncertainty_cube:
#     print('',val)
#     for valval in orbit_w_uncertainty_cube[val]:
#         print('    ', valval, '-- size: ' , np.shape(orbit_w_uncertainty_cube[val][valval] ))
#         for valvalval in orbit_w_uncertainty_cube[val][valval]:
#             print('        ', valvalval)


In [11]:
orbit_w_uncertainty_cube.keys()

dict_keys([datetime.datetime(2003, 11, 9, 0, 7, 4), datetime.datetime(2003, 11, 9, 0, 8, 4), datetime.datetime(2003, 11, 9, 0, 9, 4), datetime.datetime(2003, 11, 9, 0, 10, 4), datetime.datetime(2003, 11, 9, 0, 11, 4)])

## Next steps:

1. What data structure would work best for FORTRAN to read these tables at each timestep?
    - ~~linked list structue?~~
    - ~~a look up table?~~
    - ~~pass the dict straight to fortran?~~
    - **NETCDF file**

2. Do a full run of the INIT_ORBIT through the Kamodo Cube of Uncertainty function
    - expand the deltas in each direction so that we always have our box surrounding the orbit.

3. Plug this data structure into fortran such that at any given timestep, requested by GEODYN the value can be found in the cube.

4. Add the interpolation function to the fortran code.

5. Do a full run of Kamodo-TIEGCM with the new interpolation method 
    - there is a chance this wont work and we may need to go ahead and implement the improved grid in the cube.
    
    
Additional Features and Queries that will need to be addressed:
  - increase the resolution of the grid within the cube
      - can maybe make it match MSISe2's grid resolution
      - if we have many cubes within the overall cube surrounding our INIT_ORBIT, we should write a simple selector code block that checks which sub-cube the coordinate is in, and only do the interpolation with that sub-cube.
  - determine by how much GEODYN perturbs the coordinates on the orbit in the early iterations
  
  - how can this all be written such that the file doesn't have to be re-written but is only added to.
      - can we construct the data-structures on a grid relative to earth such that additions can be made from separate runs?




### Write dict as NETCDF file



In [168]:
TEST_NUMS = 3

We need to initialize these arrays and add on to them if necessary

Make a grid that contains the possible altitude and longitude values.  We will save the values from the orbit to the nearest point on this grid.

In [174]:
lons_array = np.arange(0, 360, 2)     # accurate to a degree
lats_array = np.arange(-90, 90, 2)    # accurate to a degree
alts_array = np.arange(440, 500, .5)  # accurate to 20 meters

print('shape lats_array',lats_array.shape)
print('shape lons_array',lons_array.shape)
print('shape alts_array',alts_array.shape)

shape lats_array (90,)
shape lons_array (180,)
shape alts_array (120,)


In [219]:
import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SingleSatelliteFlythrough import SingleModelFlythrough
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SatelliteFlythrough import ModelFlythrough


times_list = []
rhos  = np.empty((TEST_NUMS,np.size(lons_array),np.size(lats_array), np.size(alts_array)))*np.nan

for it,val in enumerate(DEN_csv['Elapsed Secs'][:TEST_NUMS]):
    ## Make each timestep a dictionary to hold our coordinates
    date_index = DEN_csv['YYMMDD'][it] + DEN_csv['HHMMSS'][it]
#     date_index = datetime.strptime(date_str, '%y%m%d%H%M%S')
    unix_time  = DEN_csv['sattime_utctimestamp'][it]
    print('****',date_index, '****' )    

    times_list.append(date_str)

    ### Get the coordinates along the orbit:
    lon = float(DEN_csv['Lon'][it])
    lat = float(DEN_csv['Lat'][it])
    alt = float(DEN_csv['Height_kilometers'][it])
    center_coord = [lon, lat, alt]
    
    
    ### Find the coordinates of the cube surround the orbit point:
    delta_deg = 2    # degrees
    delta_m = 1000.*1e-3 # meters to kilometers
    A = [lon + delta_deg, lat+delta_deg, alt+delta_m]  # top,    front, left
    B = [lon + delta_deg, lat-delta_deg, alt+delta_m]  # top,    back,  Left
    C = [lon - delta_deg, lat+delta_deg, alt+delta_m]  # top,    front, right
    D = [lon - delta_deg, lat-delta_deg, alt+delta_m]  # top,    back,  right
    E = [lon + delta_deg, lat+delta_deg, alt-delta_m]  # bottom, front, left
    F = [lon + delta_deg, lat-delta_deg, alt-delta_m]  # bottom, back,  left
    G = [lon - delta_deg, lat+delta_deg, alt-delta_m]  # bottom, front, right
    H = [lon - delta_deg, lat-delta_deg, alt-delta_m]  # bottom, back,  right
    
    
    ### Store the cube's coordinates in the dictionary index
    cube_corners_and_center = []
    cube_corners_and_center.append(center_coord)
    cube_corners_and_center.append(A)
    cube_corners_and_center.append(B)
    cube_corners_and_center.append(C)
    cube_corners_and_center.append(D)
    cube_corners_and_center.append(E)
    cube_corners_and_center.append(F)
    cube_corners_and_center.append(G)
    cube_corners_and_center.append(H)
    
    
    #### Import Coordinates to Kamodo
    ##
    #### Kamodo static inputs:
    model          = 'TIEGCM'
    file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
    variable_list  = ['rho','psi_O2', 'psi_O', 'psi_N2', 'psi_He', 'T_n']
    coord_type     = 'SPH'
    coord_grid     = 'sph'
    high_res       = 1.
    verbose        = False  
    csv_output     = '' 
    plot_output    = ''
    
    
    #### Extract the coordinates from each list to plug into Kamodo with vectorization
    lons_in = [item[0] for item in cube_corners_and_center]
    lats_in = [item[1] for item in cube_corners_and_center]
    alts_in = [item[2] for item in cube_corners_and_center]
       
    
    ## Gather inputs
    sat_time       = unix_time*np.ones(np.size(alts_in))
    c1             = lons_in
    c2             = lats_in
    c3             = alts_in
    ## Plug vectorized coordinates into Kamodo
    results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
                        coord_type, coord_grid, high_res=20., verbose=False, 
                        csv_output='', plot_output='')

    
    
    #### Save the values from Kamodo to the timestep:
    rho = [rho for rho in results['rho']]
    
    # We will loop through the values and find the gridpoint nearest to our coord. value for each dim
    print(it)
    
    
    ### We want to save the value at each corner of the cube to the grid:
    for icorner in np.arange(0,len(cube_corners_and_center)):
        
        # For each corner, get the closest index on the grid to that value for each coord.
        closestindx_lon = min(range(len(lons_array)), key = lambda i: abs(lons_array[i]-lons_in[icorner]))        
        closestindx_lat = min(range(len(lats_array)), key = lambda i: abs(lats_array[i]-lats_in[icorner]))
        closestindx_alt = min(range(len(alts_array)), key = lambda i: abs(alts_array[i]-alts_in[icorner]))
        
        
        rhos[it, closestindx_lon, closestindx_lat, closestindx_alt] = rho[i_corner]

    
                    
    
    
    
    
    
    
    
    #### Save the coordinates from RHO to the nearest gridpoint index.
#     for ilon in lons_in:
#         closestindx_lon = min(range(len(lons_array)), key = lambda i: abs(lons_array[i]-ilon))        
#         for ilat in lats_in:
#             closestindx_lat = min(range(len(lats_array)), key = lambda i: abs(lats_array[i]-ilat))
#             for ialt in alts_in:
#                 closest_altindx = min(range(len(alts_array)), key = lambda i: abs(alts_array[i]-ialt))
    
    
#                 for i_corner, corner in enumerate(rho):
                    
#                     rhos[it, closestindx_lon, closestindx_lat, closest_altindx] = rho[i_corner]
    
    
    


**** 031109000704 ****
0
**** 031109000804 ****
1
**** 031109000904 ****
2


In [178]:
rhos[it,closestindx_lon,closestindx_lat,:]

array([           nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
      

Practice making netcdf manually

In [182]:
print('lons_array',lons_array.shape)
print('lats_array',lats_array.shape)
print('alts_array',alts_array.shape)

lons_array (180,)
lats_array (90,)
alts_array (120,)


In [191]:
lats_array.shape[0]

90

In [218]:
len(times)

RuntimeError: NetCDF: Not a valid ID

In [220]:
%load_ext autoreload
%autoreload 2



import netCDF4 as nc
import numpy as np
import os
fn = 'test_manualwrite.nc'
os.remove(fn)


ds = nc.Dataset(fn, 'w', format='NETCDF4')

time = ds.createDimension('time', len(times_list))
lat = ds.createDimension('lat', int(lats_array.shape[0]))
lon = ds.createDimension('lon', int(lons_array.shape[0]))
lon = ds.createDimension('alt', int(alts_array.shape[0]))

times = ds.createVariable('time', 'f4', ('time',))
lats = ds.createVariable('lat', 'f4', ('lat',))
lons = ds.createVariable('lon', 'f4', ('lon',))
alts = ds.createVariable('alt', 'f4', ('alt',))
rho = ds.createVariable('rho', 'f4', ('time', 'lat', 'lon','alt'))
rho.units = 'g/cm^3'

lat_array = lats_array
lon_array = lons_array
alt_array = alts_array

lats[:] = lat_array
lons[:] = lon_array
alts[:] = alt_array


print('var size ', rho.shape)
# rho[0, :, :, :] = np.random.uniform(0, 100, size=(360, 720))
# print('var size after adding first data', value.shape)


ds.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
var size  (3, 90, 180, 120)


In [230]:
ds = nc.Dataset(fn)




In [231]:
ds.variables['rho'][:]



masked_array(
  data=[[[[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --]],

         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --]],

         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --]],

         ...,

         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --, --, --],
   

In [229]:
ds.

'NETCDF4'

In [201]:
# lon_array = np.arange(0, 360, 1)
# lat_array = np.arange(-90, 90, 1)


In [202]:
# print(ds.__dict__)

print('Access the dimensions:')
for dim in ds.dimensions.values():
    print('    ',dim)

    
print('Access the variable metadata:')
for var in ds.variables.values():
    print(var)
    

Access the dimensions:
     <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 0
     <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 90
     <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 180
     <class 'netCDF4._netCDF4.Dimension'>: name = 'alt', size = 120
Access the variable metadata:
<class 'netCDF4._netCDF4.Variable'>
float32 time(time)
unlimited dimensions: time
current shape = (0,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
unlimited dimensions: 
current shape = (90,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
unlimited dimensions: 
current shape = (180,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float32 alt(alt)
unlimited dimensions: 
current shape = (120,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCD

Build a single matrix containing these values

(3,)

In [84]:
# d = {}
# d['time'] = ('time',times)
# d['latitudes'] = ('latitudes',lats)
# d['longitudes'] = ('longitudes', lons)
# d['altitudes'] = ('altitudes', alts)
# d['rhos'] = (['time'], rhos)


In [85]:
# dxr = xr.DataArray(d)

In [86]:
# dxr.to_netcdf(path='test_file.nc', mode='w')

In [92]:
ds = nc.Dataset(fn, 'r', format='NETCDF4')
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 

Using multi-indexing we can make a series of our rho values

Below is the datastructure as a pandas df with multiindexing

In [ ]:

for it, val_t in enumerate(times):
    idx = pd.MultiIndex.from_arrays(arrays=[lats[it],lons[it], alts[it]], names=["lat","lon","alts"])
    s = pd.Series(data=rhos[it], index=idx)
    
# use from_series method
#     da = xr.DataArray.from_series(s)
# da

In [63]:
# idx = pd.MultiIndex.from_arrays(arrays=[[], [],[], []], names=["time","lat","lon","alts"])

idx = pd.DataFrame(index = [['time','time','time','time','time','time','time','time','time'],
                                           ['center','A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], ],
                                columns = ['lon', 'lat', 'alt', 'rho'],
                               )
idx
# s = pd.Series(data='', index=idx)


# df = pd.DataFrame(s)
# df

lon  lat  alt  rho
time center  NaN  NaN  NaN  NaN
     A       NaN  NaN  NaN  NaN
     B       NaN  NaN  NaN  NaN
     C       NaN  NaN  NaN  NaN
     D       NaN  NaN  NaN  NaN
     E       NaN  NaN  NaN  NaN
     F       NaN  NaN  NaN  NaN
     G       NaN  NaN  NaN  NaN
     H       NaN  NaN  NaN  NaN

In [ ]:
sys.exit('STOP RUNNING BEFORE THE TESTS')

# Tests and Attempts Below

### Test if cube works for a set of noisy data

In [ ]:
import random

lons = DEN_csv['Lon'][:TEST_NUMS]
lats = DEN_csv['Lat'][:TEST_NUMS]
alts = DEN_csv['Height_kilometers'][:TEST_NUMS]       
noisy_lons = [i+random.randint(-1,1) for i in lons]
noisy_lats = [i+random.randint(-1,1) for i in lats]
noisy_alts = [i+random.randint(-1,1) for i in alts]

In [ ]:
kamodo_direct = []
interpd_orbit = []

for i,val in enumerate(DEN_csv['Elapsed Secs'][:TEST_NUMS]):
    
    print('****',date_index, '****' )
    ## Make each timestep a dictionary to hold our coordinates
    date_index = DEN_csv['YYMMDD'][i] + DEN_csv['HHMMSS'][i]
    unix_time  = DEN_csv['sattime_utctimestamp'][i]
    
    test_lon = noisy_lons[i]
    test_lat = noisy_lats[i]
    test_alt = noisy_alts[i]
    
    sat_time       = [unix_time]
    c1             = [test_lon]
    c2             = [test_lat]
    c3             = [test_alt]

    results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
                        coord_type, coord_grid, high_res=20., verbose=False, 
                        csv_output='', plot_output='')
    
#     print('      ',results['rho'][0] )
#     print('      ',interpolate_point_in_cube(test_lon, test_lat, test_alt, uncertain_grid, date_index, delta_deg, delta_m)[0])
    kamodo_direct.append(results['rho'][0])
    interpd_orbit.append(interpolate_point_in_cube(test_lon, test_lat, test_alt, uncertain_grid, date_index, delta_deg, delta_m)[0])

    
    
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px


config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })

fig = make_subplots(rows=1, cols=1,
                   )


fig.add_trace(go.Scatter(x=DEN_csv['sattime_utctimestamp'][:TEST_NUMS].values,
                         y=kamodo_direct,
                         mode='markers',
                         marker=dict(size=3)),
                         )

fig.add_trace(go.Scatter(x=DEN_csv['sattime_utctimestamp'][:TEST_NUMS].values,
                         y=interpd_orbit,
                         mode='markers',
                         marker=dict(size=3)),
                         )

fig.show()

In [ ]:
# %load_ext autoreload
# %autoreload 2
# from numpy import array
# import sys
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# from SingleSatelliteFlythrough import SingleModelFlythrough
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# from SatelliteFlythrough import ModelFlythrough



# #### Kamodo static inputs:
# model          = 'TIEGCM'
# file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
# variable_list  = ['rho','psi_O2', 'psi_O', 'psi_N2', 'psi_He', 'T_n']
# coord_type     = 'SPH'
# coord_grid     = 'sph'
# high_res       = 1.
# verbose        = False  
# csv_output     = '' 
# plot_output    = ''

    
# for i,date in enumerate(uncertain_grid['YYMMDDHHMMSS']):
#     print('***** '+ date +' *****')
    
#     for ii,loop in enumerate(loops):

#         sat_time       = [uncertain_grid['sattime_utctimestamp'][i]]
#         c1             = [uncertain_grid['lon'+loop[0]][i]]
#         c2             = [uncertain_grid['lat'+loop[1]][i]]
#         c3             = [uncertain_grid['z'+loop[2]][i]]
#         print('    ','lon'+loop[0],'lat'+loop[1] , 'z'+loop[2] )
        
#         results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
#                             coord_type, coord_grid, high_res=20., verbose=False, 
#                             csv_output='', plot_output='')



In [ ]:
# ########################## CHECK WITH A PLOT:

# import plotly.graph_objects as go
# import numpy as np


# fig = go.Figure(data=[go.Scatter3d(x=[lon], y=[lat], z=[alt],
#                                    mode='markers')])
# ms_size = 10
# fig.add_trace(go.Scatter3d(
#                          x=[A[0]], y=[A[1]], z=[A[2]],
#                          mode='markers',
#                         name = 'A',
#                          marker=dict(size=ms_size)),
#                          )
# fig.add_trace(go.Scatter3d(
#                          x=[B[0]], y=[B[1]], z=[B[2]],
#                          mode='markers',
#                         name = 'B',
#                          marker=dict(size=ms_size)),
#                          )
# fig.add_trace(go.Scatter3d(
#                          x=[C[0]], y=[C[1]], z=[C[2]],
#                          mode='markers',
#                         name = 'C',
#                          marker=dict(size=ms_size)),
#                          )
# fig.add_trace(go.Scatter3d(
#                          x=[D[0]], y=[D[1]], z=[D[2]],
#                          mode='markers',
#                         name = 'D',
#                          marker=dict(size=ms_size)),
#                          )
# fig.add_trace(go.Scatter3d(
#                          x=[E[0]], y=[E[1]], z=[E[2]],
#                          mode='markers',
#                         name = 'E',
#                          marker=dict(size=ms_size)),
#                          )
# fig.add_trace(go.Scatter3d(
#                          x=[F[0]], y=[F[1]], z=[F[2]],
#                          mode='markers',
#                         name = 'F',
#                          marker=dict(size=ms_size)),
#                          )
# fig.add_trace(go.Scatter3d(
#                          x=[G[0]], y=[G[1]], z=[G[2]],
#                          mode='markers',
#                         name = 'G',
#                          marker=dict(size=ms_size)),
#                          )
# fig.add_trace(go.Scatter3d(
#                          x=[H[0]], y=[H[1]], z=[H[2]],
#                          mode='markers',
#                         name = 'H',
#                          marker=dict(size=ms_size)),
#                          )
# fig.show()    

### Attempt for all permutations of lonlatz 

In [ ]:
uncertain_grid = {}


delta_deg = 0.5 # degrees
delta_met = 10.*1e-3 # meters to kilometers

uncertain_grid['YYMMDDHHMMSS']  = []
uncertain_grid['sattime_utctimestamp']  = []
uncertain_grid['lat']           = []
uncertain_grid['lat_p']         = []
uncertain_grid['lat_m']         = []
uncertain_grid['lon']           = []
uncertain_grid['lon_p']         = []
uncertain_grid['lon_m']         = []
uncertain_grid['z']             = []
uncertain_grid['z_p']           = []
uncertain_grid['z_m']           = []


for i,val in enumerate(DEN_csv['Elapsed Secs'][:20]):
    uncertain_grid['YYMMDDHHMMSS'].append(DEN_csv['YYMMDD'][i] + DEN_csv['HHMMSS'][i])
    uncertain_grid['sattime_utctimestamp'].append(DEN_csv['sattime_utctimestamp'][i])
    
    #### Perturbate the coordinates to some uncertainty
    uncertain_grid['lat'].append(DEN_csv['Lat'][i])
    uncertain_grid['lat_p'].append(DEN_csv['Lat'][i] + delta_deg)
    uncertain_grid['lat_m'].append(DEN_csv['Lat'][i] - delta_deg)
    
    uncertain_grid['lon'].append(DEN_csv['Lon'][i])
    uncertain_grid['lon_p'].append(DEN_csv['Lon'][i] + delta_deg)
    uncertain_grid['lon_m'].append(DEN_csv['Lon'][i] - delta_deg)
    
    uncertain_grid['z'].append(DEN_csv['Height_kilometers'][i])
    uncertain_grid['z_p'].append(DEN_csv['Height_kilometers'][i] + delta_met)
    uncertain_grid['z_m'].append(DEN_csv['Height_kilometers'][i] - delta_met)
    
    

%load_ext autoreload
%autoreload 2
from numpy import array
import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SingleSatelliteFlythrough import SingleModelFlythrough
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SatelliteFlythrough import ModelFlythrough
import itertools

# for combo in itertools.combinations_with_replacement(['', '_p', '_m'], 3):
#     loop_options.append([i + j for i, j in zip(['', '', ''], combo)])

def perm(n, seq):
    loop_options = []
    for p in itertools.product(seq, repeat=n):
        loop_options.append([i + j for i, j in zip(['', '', ''], p)])
    return(loop_options)
loops = perm(3, ['', '_m', '_p'])


#### Kamodo static inputs:
model          = 'TIEGCM'
file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
variable_list  = ['rho','psi_O2', 'psi_O', 'psi_N2', 'psi_He', 'T_n']
coord_type     = 'SPH'
coord_grid     = 'sph'
high_res       = 1.
verbose        = False  
csv_output     = '' 
plot_output    = ''



#### Open file to construct CSV
# CSV_MAKE_FLAG = True
# if CSV_MAKE_FLAG:

#     csv_filename = 'orbit_uncertainty_cloud_'+'.csv'
#     data_out = open(csv_filename, 'w')
#     data_out.write(f'\n  #Model: {model} \n')
#     data_out.write(f'\n  YYMMDDHHMMSS  Unix time     Lon      Lat    Altitude[km]       Density         \n')
#                         #031109000704 1068336424.0 217.8688 -25.1728 1113.435865 1.8578944895229333e-19

    
for i,date in enumerate(uncertain_grid['YYMMDDHHMMSS']):
    print('***** '+ date +' *****')
    
    for ii,loop in enumerate(loops):

        sat_time       = [uncertain_grid['sattime_utctimestamp'][i]]
        c1             = [uncertain_grid['lon'+loop[0]][i]]
        c2             = [uncertain_grid['lat'+loop[1]][i]]
        c3             = [uncertain_grid['z'+loop[2]][i]]
        print('    ','lon'+loop[0],'lat'+loop[1] , 'z'+loop[2] )
        
        results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
                            coord_type, coord_grid, high_res=20., verbose=False, 
                            csv_output='', plot_output='')

#         if CSV_MAKE_FLAG:
#             for ii,values in enumerate(results['utc_time']):
#                 data_out.write( uncertain_grid['YYMMDDHHMMSS'][i]+' '+
#                                 str(results['utc_time'][ii]) +' '+
#                                 str(c1[0]) +' '+
#                                 str(c2[0]) +' '+
#                                 str(c3[0]) +' '+
#                                 #
#                                 str(results['rho'][0])  
#                                 +'\n')
data_out.close()


Try making a grid in the the altitude dimension only

In [ ]:
sys.exit()

In [ ]:
%load_ext autoreload
%autoreload 2
from numpy import array
import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SingleSatelliteFlythrough import SingleModelFlythrough
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SatelliteFlythrough import ModelFlythrough




uncertain_grid = {}

CSV_MAKE_FLAG = True
delta_deg = 0.5 # degrees
delta_met = 10.*1e-3 # meters to kilometers

uncertain_grid['YYMMDDHHMMSS']  = []
uncertain_grid['sattime_utctimestamp']  = []
uncertain_grid['z']             = []
uncertain_grid['z_p']           = []
uncertain_grid['z_m']           = []
uncertain_grid['z_2p']           = []
uncertain_grid['z_2m']           = []
uncertain_grid['lon']             = []
uncertain_grid['lon_p']           = []
uncertain_grid['lon_m']           = []
uncertain_grid['lon_2p']           = []
uncertain_grid['lon_2m']           = []
uncertain_grid['lat']             = []
uncertain_grid['lat_p']           = []
uncertain_grid['lat_m']           = []
uncertain_grid['lat_2p']           = []
uncertain_grid['lat_2m']           = []


for i,val in enumerate(DEN_csv['Elapsed Secs'][:20]):
    uncertain_grid['YYMMDDHHMMSS'].append(DEN_csv['YYMMDD'][i] + DEN_csv['HHMMSS'][i])
    uncertain_grid['sattime_utctimestamp'].append(DEN_csv['sattime_utctimestamp'][i])
    uncertain_grid['lat'].append(DEN_csv['Lat'][i])
    uncertain_grid['lon'].append(DEN_csv['Lon'][i])

    #### Perturbate the coordinates to some uncertainty
    uncertain_grid['lon'].append(DEN_csv['Lon'][i])
    uncertain_grid['lon_p'].append(DEN_csv['Lon'][i] + delta_deg)
    uncertain_grid['lon_m'].append(DEN_csv['Lon'][i] - delta_deg)
    uncertain_grid['lon_2p'].append(DEN_csv['Lon'][i] + 2*delta_deg)
    uncertain_grid['lon_2m'].append(DEN_csv['Lon'][i] - 2*delta_deg)
    
    uncertain_grid['lat'].append(DEN_csv['Lat'][i])
    uncertain_grid['lat_p'].append(DEN_csv['Lat'][i] + delta_deg)
    uncertain_grid['lat_m'].append(DEN_csv['Lat'][i] - delta_deg)
    uncertain_grid['lat_2p'].append(DEN_csv['Lat'][i] + 2*delta_deg)
    uncertain_grid['lat_2m'].append(DEN_csv['Lat'][i] - 2*delta_deg)
    
    uncertain_grid['z'].append(DEN_csv['Height_kilometers'][i])
    uncertain_grid['z_p'].append(DEN_csv['Height_kilometers'][i] + delta_met)
    uncertain_grid['z_m'].append(DEN_csv['Height_kilometers'][i] - delta_met)
    uncertain_grid['z_2p'].append(DEN_csv['Height_kilometers'][i] + 2*delta_met)
    uncertain_grid['z_2m'].append(DEN_csv['Height_kilometers'][i] - 2*delta_met)


# import itertools
# def perm(n, seq):
#     loop_options = []
#     for p in itertools.product(seq, repeat=n):
#         loop_options.append([i + j for i, j in zip(['', '', ''], p)])
#     return(loop_options)
# loops = perm(3, ['', '_m', '_p'])


### OPEN CSV FIRST outside of the loop:
CSV_MAKE_FLAG = True

if CSV_MAKE_FLAG:

    csv_filename = 'orbit_uncertainty_cloud_'+'.csv'
    data_out = open(csv_filename, 'w')
    data_out.write(f'\n#Model used:, {file_dir}')



for i,date in enumerate(uncertain_grid['YYMMDDHHMMSS']):
    print('***** '+ date +' *****')
    model          = 'TIEGCM'
    file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
    variable_list  = ['rho','psi_O2', 'psi_O', 'psi_N2', 'psi_He', 'T_n']
    coord_type     = 'SPH'
    coord_grid     = 'sph'
    high_res       = 1.
    verbose        = False  
    csv_output     = '' 
    plot_output    = ''
    
#     c1             = [uncertain_grid['lon_2m'][i], 
#                       uncertain_grid['lon_m'][i],
#                       uncertain_grid['lon'][i],
#                       uncertain_grid['lon_p'][i],
#                       uncertain_grid['lon_2p'][i] ]
# --------------------------------------------------------
# --------------------------------------------------------
#     c2             = [uncertain_grid['lat_2m'][i], 
#                       uncertain_grid['lat_m'][i],
#                       uncertain_grid['lat'][i],
#                       uncertain_grid['lat_p'][i],
#                       uncertain_grid['lat_2p'][i] ]

    c3             = [uncertain_grid['z_2m'][i], 
                      uncertain_grid['z_m'][i],
                      uncertain_grid['z'][i],
                      uncertain_grid['z_p'][i],
                      uncertain_grid['z_2p'][i] ]
    c1             = uncertain_grid['lon'][i]*np.ones(np.size(c3))
    c2             = uncertain_grid['lat'][i]*np.ones(np.size(c3))

    sat_time       = uncertain_grid['sattime_utctimestamp'][i]*np.ones(np.size(c3))

    results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
                        coord_type, coord_grid, high_res=20., verbose=False, 
                        csv_output='', plot_output='')
    
    if CSV_MAKE_FLAG:
        for ii,values in enumerate(results['utc_time']):
            data_out.write( uncertain_grid['YYMMDDHHMMSS'][i]+' '+
                            str(results['utc_time'][ii]) +' '+
                            str(results['c1'][ii]) +' '+
                            str(results['c2'][ii]) +' '+
                            str(uncertain_grid['z_2m'][ii]) +' '+
                            str(uncertain_grid['z_m'][ii]) +' '+
                            str(uncertain_grid['z'][ii]) +' '+
                            str(uncertain_grid['z_p'][ii]) +' '+
                            str(uncertain_grid['z_2p'][ii]) +' '+
                            #
                            str(results['rho'][0]) +' '+
                            str(results['rho'][1]) +' '+
                            str(results['rho'][2]) +' '+
                            str(results['rho'][3]) +' '+
                            str(results['rho'][4]) 
                            +'\n')
data_out.close()



In [ ]:
results

In [ ]:
# import itertools 
# A = list(itertools.product(['1', '2', '3'],['', '_p', '_m']))

# list_join = []
# for i in A:
#     join_item = ''
#     list_join.append(join_item.join(i))
# # print(list_join)

# B = list(itertools.product(['lon', 'lat', 'alt'],list_join))
# print(B)


In [ ]:
loop_options = []
# for combo in itertools.combinations_with_replacement(['1', '2', '3'],3):
for combo in itertools.permutations(['', '_m', '_p'],3):

    loop_options.append([i + j for i, j in zip(['', '', ''], combo)])

(loop_options)

In [ ]:
import itertools
def perm(n, seq):
    count = 0

    for p in itertools.product(seq, repeat=n):
#         print((p), count)
        count+=1
        print(count, [i + j for i, j in zip(['lon', 'lat', 'alt'], p)])
perm(3, ['', '_m', '_p'])

In [ ]:
DEN_csv

In [ ]:
1.7*40

In [ ]:
%load_ext autoreload
%autoreload 2
from numpy import array
import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SingleSatelliteFlythrough import SingleModelFlythrough

sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SatelliteFlythrough import ModelFlythrough


model          = 'TIEGCM'
file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
variable_list  = ['rho','psi_O2', 'psi_O', 'psi_N2', 'psi_He', 'T_n']
c3             = DEN_csv['Height (meters)'].astype(float)[:]
sat_time       = DEN_csv['sattime_utctimestamp'].astype(float)[:]
c1             = DEN_csv['Lon'].astype(float)[:]
c2             = DEN_csv['Lat'].astype(float)[:]
coord_type      = 'SPH'  # zach left this as sph because need lon to be mapped to -180-180   
coord_grid      = 'sph'    
high_res       = 1.
verbose        = False  
csv_output      ='' 
plot_output     =''
 
results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
                    coord_type, coord_grid, high_res=20., verbose=False, 
                    csv_output='', plot_output='')



## Do some tests to make sure Kamodo Extrapolation is working



### Kamodo full satellite flythrough

In [ ]:
sys.exit()

In [ ]:
type(DEN_csv['Lon'].values[0])

In [ ]:
from netCDF4 import Dataset
import numpy as np 
import os
import warnings
warnings.filterwarnings("ignore")

def read_tiegcm( filename, variables):
    ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
        The breakloop feature is here so that if the file doesn't exist the code can still continue if placed in a loop.'''
    status = os.path.exists(filename)
 
    if status == True:
        data = {}
        for i, var_names in enumerate(variables):
            ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
            varData = ncid.variables
            data[var_names] = np.array(varData[var_names])  
    elif status == False:
        print('No File Found', filename )
        breakloop = True
        data = 0
        return( data , breakloop)
    breakloop = False
    return(data,breakloop )


def find_modelgrid_base_index(TIEGCM, lon_sat, lat_sat, time_sat):
    """ 
    Interpolation function: (3 / 5)

    This function locates the grid points that make up thr square surrounding out desired point.
    """

    lon_start = -180 # first longitude point of the model grid (deg)
    dlon = 5  # difference between adjacent longitude grid points (deg)
    index_lon0 = int(np.mod(np.floor( (lon_sat - lon_start )/dlon ), 72 ) ) 


    lat_start = -87.5 # first longitude point of the model grid (deg)
    dlat = 5 # difference between adjacent longitude grid points (deg)
    index_lat0 = int(np.mod(np.floor( (lat_sat - lat_start)/dlat ), 36 ) )

    time_start = 0 # first longitude point of the model grid (deg)
    dtime = 1 # difference between adjacent longitude grid points (deg)
    index_time = int((np.mod(np.floor( (time_sat - time_start)/dtime ), 24 ) )) -1

    if index_lon0 == 71:
        index_lon1 = 0
    else:
        index_lon1 = index_lon0 + 1


    if index_lat0 >= 35 :
        index_lat1 = 35
        NPole_Flag = True
        SPole_Flag = False 

    elif index_lat0 <= 0: 
        index_lat1 = 0
        SPole_Flag = True
        NPole_Flag = False
    else: 
        index_lat1 = index_lat0 + 1
        NPole_Flag = False 
        SPole_Flag = False 


    return(index_lon0, index_lat0, index_lon1, index_lat1, index_time, NPole_Flag, SPole_Flag )


def DEN_and_ZG_vertprofs(TIEGCM, lon_sat, lat_sat, time_sat, param):
    '''
    Interpolation function: (2 / 5)

    This function finds the vertical profiles of the four grid points around a desired point.
    '''
    
    #### First we find the four grid points:
    (indexlon0, 
    indexlat0, 
    indexlon1, 
    indexlat1, 
    indexut, 
    NPole_Flag, 
    SPole_Flag) = find_modelgrid_base_index(TIEGCM, lon_sat, lat_sat, time_sat)
    grid_vals = pd.DataFrame(data = {'lon0' : TIEGCM['lon'][indexlon0] ,\
                                 'lonindex0': indexlon0 ,\
                                 'lon1'     :TIEGCM['lon'][indexlon1] ,\
                                 'lonindex1':indexlon1 ,\
                                 'lat0'     :TIEGCM['lat'][indexlat0] ,\
                                 'latindex0':indexlat0 ,\
                                 'lat1'     :TIEGCM['lat'][indexlat1] ,\
                                 'latindex1':indexlat1  }, index=[0])
#     print('UT index',indexut)
#     print('UT value',TIEGCM['ut'][indexut])
#     print()
#     print('Lon0 index',indexlon0)
#     print('Lon0 value',TIEGCM['lon'][indexlon0])
#     print('Lon1 index',indexlon1)
#     print('Lon1 value',TIEGCM['lon'][indexlon1])
#     print()
#     print('Lat0 index',indexlat0)
#     print('Lat0 value',TIEGCM['lat'][indexlat0])
#     print('Lat1 index',indexlat1)
#     print('Lat1 value',TIEGCM['lat'][indexlat1])
    
#     print('alt index',indexut)
#     print('alt value',TIEGCM['ut'][indexut])

    DEN_prof00 = TIEGCM[param][indexut, :-1, indexlat0, indexlon0]
    DEN_prof10 = TIEGCM[param][indexut, :-1, indexlat1, indexlon0]
    DEN_prof01 = TIEGCM[param][indexut, :-1, indexlat0, indexlon1]
    DEN_prof11 = TIEGCM[param][indexut, :-1, indexlat1, indexlon1]
    

    ZG_prof00 = TIEGCM['ZG'][indexut, :-1, indexlat0, indexlon0]
    ZG_prof10 = TIEGCM['ZG'][indexut, :-1, indexlat1, indexlon0]
    ZG_prof01 = TIEGCM['ZG'][indexut, :-1, indexlat0, indexlon1]
    ZG_prof11 = TIEGCM['ZG'][indexut, :-1, indexlat1, indexlon1]

    DEN_df = pd.DataFrame(data = {'00' :DEN_prof00 ,\
                                  '10' :DEN_prof10 ,\
                                  '01' :DEN_prof01 ,\
                                  '11' :DEN_prof11 ,\
                                })
    ZG_df = pd.DataFrame(data = {'00' :ZG_prof00 ,\
                                  '10' :ZG_prof10 ,\
                                  '01' :ZG_prof01 ,\
                                  '11' :ZG_prof11 ,\
                                })
    return(DEN_df, ZG_df, grid_vals)



def interp_DENSITY_to_altitude(DEN_df, ZG_df, height_sat):
    '''
    Interpolation function: (4 / 5)

    This function interpolates Density in the log space then converts it back'''
    density_surf_at_height_sat = np.zeros(np.size(ZG_df.columns))
    i = 0
    for col in ZG_df.columns:
        xp = ZG_df[col][:].values*1e-5
        fp = DEN_df[col][:].values
        xval = height_sat
        density_surf_at_height_sat[i] =  np.exp( np.interp(xval, xp, np.log(fp))  )
        i += 1
    return(density_surf_at_height_sat)



def interp_param_to_altitude(df, ZG_df, height_sat):
    
    '''
    Interpolation function: (5 / 5)

    This function interpolates all other parameters (not density) using linear interpolation
    '''

    param_surf_at_height_sat = np.zeros(np.size(ZG_df.columns))
    i = 0
    for col in ZG_df.columns:
        xp = ZG_df[col][:].values*1e-5
        fp = df[col][:].values
        xval = height_sat
        param_surf_at_height_sat[i] =  np.interp(xval, xp, fp)  
        i += 1
    return(param_surf_at_height_sat)





def interpolate_tiegcm(TIEGCM, lon_sat, lat_sat, time_sat, height_sat, param):
    '''
    Interpolation function: (1 / 5)
    
    This function finds the interpolated value of a TIEGCM parameter.  
    Interpolated to a desired lon,lat,alt
    '''
    
    # Find the vertical profiles of the four grid points making up the grid square around your desired point.
#     print('time_sat:', time_sat)
    df, ZG_df, grid_vals = DEN_and_ZG_vertprofs(TIEGCM, lon_sat, lat_sat, time_sat, param)
    
    if param == 'DEN':
        param_surf_at_height_sat = interp_DENSITY_to_altitude(df, ZG_df, height_sat)
    else:
        param_surf_at_height_sat = interp_param_to_altitude(df, ZG_df, height_sat)


    xval = lon_sat

    x_lon = [grid_vals['lon0'][0], grid_vals['lon1'][0]]
    f_den0 = [param_surf_at_height_sat[0], param_surf_at_height_sat[1]]
    interp_lon0 =   np.interp(xval, x_lon, f_den0  )

    f_den1 = [param_surf_at_height_sat[2], param_surf_at_height_sat[3]]
    interp_lon1 =   np.interp(xval, x_lon, f_den1  )

    final_interped_value = interp_lon0 + ((lat_sat - grid_vals['lat0'][0]) / (grid_vals['lat1'][0] - grid_vals['lat0'][0]))*(interp_lon1 - interp_lon0)

    return(final_interped_value)


# def gravity(h):
#     """For h in same units as R_e  (computes in cm as artifact of old code)"""
#     return g_0*pow(R_e/(R_e + h), 2)



# def interp_to_altitude(DEN_df, ZG_df, height_sat):
#     density_surf_at_height_sat = np.zeros(np.size(ZG_df.columns))
#     i = 0
#     for col in ZG_df.columns:
#         xp = ZG_df[col][:].values*1e-5
#         fp = DEN_df[col][:].values
#         xval = height_sat
#         density_surf_at_height_sat[i] =  np.exp( np.interp(xval, xp, np.log(fp))  )
#         i += 1
#     return(density_surf_at_height_sat)




# def interp_lon_lat(TIEGCM, lon_sat, lat_sat, time_sat, height_sat, param):
#     DEN_df, ZG_df, grid_vals = DEN_and_ZG_vertprofs(TIEGCM, lon_sat, lat_sat, time_sat, param)
#     density_surf_at_height_sat = interp_to_altitude(DEN_df, ZG_df, height_sat)

#     xval = lon_sat

#     x_lon = [grid_vals['lon0'][0], grid_vals['lon1'][0]]
#     f_den0 = [density_surf_at_height_sat[0], density_surf_at_height_sat[1]]
#     interp_lon0 =   np.interp(xval, x_lon, f_den0  )

    
#     f_den1 = [density_surf_at_height_sat[2], density_surf_at_height_sat[3]]
#     interp_lon1 =   np.interp(xval, x_lon, f_den1  )
    
#     density = interp_lon0 + ((lat_sat - grid_vals['lat0'][0]) / (grid_vals['lat1'][0] - grid_vals['lat0'][0]))*(interp_lon1 - interp_lon0)
# #     print(density)
#     return(density)

In [ ]:
%load_ext autoreload
%autoreload 2
from numpy import array
import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SingleSatelliteFlythrough import SingleModelFlythrough

sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SatelliteFlythrough import ModelFlythrough


model          = 'TIEGCM'
file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
variable_list  = ['rho','psi_O2', 'psi_O', 'psi_N2', 'psi_He', 'T_n']
c3             = DEN_csv['Height (meters)'].astype(float)[:]
sat_time       = DEN_csv['sattime_utctimestamp'].astype(float)[:]
c1             = DEN_csv['Lon'].astype(float)[:]
c2             = DEN_csv['Lat'].astype(float)[:]
coord_type      = 'SPH'  # zach left this as sph because need lon to be mapped to -180-180   
coord_grid      = 'sph'    
high_res       = 1.
verbose        = False  
csv_output      ='' 
plot_output     =''


results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
                    coord_type, coord_grid, high_res=20., verbose=False, 
                    csv_output='', plot_output='')

# print(results['c3'])
# print( results['rho'])

### Read Maually by directly indexing TIEGCM

In [ ]:
variables =['lon','lat' ,'ut','ilev','f107d','f107a','Kp', 'O2', 'O1', 'N2', 'HE', 'DEN', 'TN', 'Z', 'ZG', 'ZGMID']
MANUAL_list = []

alts = np.linspace(100, 1000, num=500)

for i,val in enumerate(DEN_csv['sattime_utctimestamp'].astype(float)[:500]) :  #calls_csv['Num'][:2]):
    kamodo_program_path = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/SingleSatelliteFlythrough.py'
    model     =  "4" #'4' #'TIEGCM'
    file_dir  = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
    sat_time1 = DEN_csv['YYMMDD'][i]  #str(calls_csv['YYMMDD'][i])   
    sat_time2 = DEN_csv['HHMMSS'][i]  #1068897600         #str(calls_csv['HHMMSS'][i])   
    c1        = DEN_csv['Lon'][i]  #'100.0'         #str(calls_csv['lon'][i])    # lon
    c2        = DEN_csv['Lat'][i]  #'32.5'         #str(calls_csv['lat'][i])   # lat
    c3        = DEN_csv['Height (meters)'][i]  #'400'   # alt
    
    tiegcm_file = '/data/data_geodyn/atmos_models_data/tiegcm/' + '%d/' % 2003  +'s%03d.nc' % 319   
    tiegcm = read_tiegcm(tiegcm_file, variables)
    tiegcm  = tiegcm[0]

    lon  = float(c1)
    lat  = float(c2)
    alt  = float(c3)
    time_hour = datetime.strptime(sat_time1+sat_time2, '%y%m%d%H%M%S').hour
    MANUAL_list.append(interpolate_tiegcm(tiegcm, lon, lat, time_hour, alt, 'DEN'))


In [ ]:
results.keys()

### Read MSIS2 with DEN_csv indices

In [ ]:


# import pandas as pd
msisin_den_file = '/data/data_geodyn/results/st/msis2/msis2_acceloffSLR_kamodo_interface/DENSITY/' + "st031109_2wk.goco05s_msisin"

DEN1_csv = pd.read_csv(msisin_den_file, 
                    skiprows = 1, 
                    names = ['IYYDDD',
                             'IYR',
                              'DAY',
                             'UTSEC',
                             'ALTKM',
                             'GLAT',
                             'GLON',
                             'STLOC', 
                             'AVGFLX',
                             'FLUX',
                             'AP1',
                             'AP2',
                             'AP3',
                             'AP4',
                             'AP5',
                             'AP6',
                             'AP7',
                            ],
                    sep = '\s+',
                    )


DEN1_csv['Date'] = (pd.to_datetime('0'+ ((DEN1_csv['IYR'].astype(int).astype(str))),  format='%y') 
                    +  pd.to_timedelta(DEN1_csv['DAY'], unit='days'))



SWI_option = [1.0]*25
SWI_option[8] = -1.0


import matplotlib.pyplot as plt
import numpy as np
from pymsis import msis
import pandas as pd


msis2_df= pd.DataFrame(data={'Rho'   :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                               'N2'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                               'O2'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                               'O'     :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                               'He'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                               'H'     :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                               'Ar'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                               'N'     :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                               'AnomO' :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                               'NO'    :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
#                               'Temp'  :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                            })


for index, row in DEN1_csv[:500].iterrows():

    lon = DEN_csv['Lon'][index] #row['GLON']
    lat = DEN_csv['Lat'][index] #row['GLAT']
    alts = DEN_csv['Height (meters)'][index] #row['ALTKM']
    f107 = DEN_csv['flux_daily'][index] # row['FLUX'] 
    f107a = DEN_csv['flux_avg'][index] # row['AVGFLX']

    aps = [[row['AP1'],row['AP2'],row['AP3'],row['AP4'],row['AP5'],row['AP6'],row['AP7']]]
    date = DEN_csv['Date'][index]  #row['Date']

    msis_data2 =  msis.run(date, lon, lat, alts, f107, f107a, aps, version=2,  options = SWI_option)
    msis_data2  = np.squeeze(msis_data2)


    msis2_df.loc[index, 'Rho'] = msis_data2[0]
#     msis2_df.loc[index, 'N2'] = msis_data2[1]
#     msis2_df.loc[index, 'O2'] = msis_data2[2]
#     msis2_df.loc[index, 'O'] = msis_data2[3]
#     msis2_df.loc[index, 'He'] = msis_data2[4]
#     msis2_df.loc[index, 'H'] = msis_data2[5]
#     msis2_df.loc[index, 'Ar'] = msis_data2[6]
#     msis2_df.loc[index, 'N'] = msis_data2[7]
#     msis2_df.loc[index, 'AnomO'] = msis_data2[8]
#     msis2_df.loc[index, 'NO'] = msis_data2[9]
#     msis2_df.loc[index, 'Temp'] = msis_data2[10]
    msis2_df.loc[index, 'Date'] = row['Date']

#     return(msis00_df, msis2_df)

#################### MAKE PLOT ####################

import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px


config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })

fig = make_subplots(rows=1, cols=1,
                   )
fig.add_trace(go.Scatter(x=msis2_df['Date'][:500],
                         y=msis2_df['Rho'][:500],
                         name= 'MSIS2 direct read',
                         mode='markers',
                         marker=dict(size=8)),
                         )
fig.add_trace(go.Scatter(y=DEN_csv['rho (kg/m**3)'][:500],
                         x=DEN_csv['Date'][:500],
                         name= 'Density File (GEODYN_msis2)',
                         mode='markers',
                        marker=dict(size=4)),
                         )
                        
# fig.add_trace(go.Scatter(y=results['max_height']*1e-3, x=results['rho'], mode='markers'))

fig.update_yaxes(type="log", exponentformat= 'power', row=1, col=1, title ='Den')
fig.update_xaxes(title ='Date')
fig.update_layout(title="MSIS2 Direct Read, Comparison of Values")

fig.show()

In [ ]:

import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px


config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })

fig = make_subplots(rows=1, cols=1,
                   )


fig.add_trace(go.Scatter(y=results['rho'],
                         x=results['utc_time'],
                         name= 'Kamodo w/ Extrapolation',
                         mode='markers+lines',
                         ),
                         )

fig.add_trace(go.Scatter(x=DEN_csv['sattime_utctimestamp'][:500],
                         y=msis2_df['Rho'][:500]*1e-3,
                         name= 'MSIS2 direct read',
                         mode='markers',
                         marker=dict(size=3)),
                         )

# fig.add_trace(go.Scatter(y=results['max_height']*1e-3, x=results['rho'], mode='markers'))

fig.update_yaxes(type="log", exponentformat= 'power', row=1, col=1, title ='Density [g/cm^3]')
fig.update_xaxes(type="linear", exponentformat= 'power', row=1, col=1, title ='timestamp')
fig.update_layout(title="(Orig Altitude) Comparing the Densities across the model options")

fig.show()

### Run the Kamodo and MSIS runs at a lower more reasonable altitude

In [ ]:
subtract_alt = 500

In [ ]:
%load_ext autoreload
%autoreload 2
from numpy import array
import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SingleSatelliteFlythrough import SingleModelFlythrough

sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SatelliteFlythrough import ModelFlythrough


model          = 'TIEGCM'
file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
variable_list  = ['rho','psi_O2', 'psi_O', 'psi_N2', 'psi_He', 'T_n']
c3             = DEN_csv['Height (meters)'].astype(float)[:500]-subtract_alt
sat_time       = DEN_csv['sattime_utctimestamp'].astype(float)[:500]
c1             = DEN_csv['Lon'].astype(float)[:500]
c2             = DEN_csv['Lat'].astype(float)[:500]
coord_type      = 'SPH'  # zach left this as sph because need lon to be mapped to -180-180   
coord_grid      = 'sph'    
high_res       = 1.
verbose        = False  
csv_output      ='' 
plot_output     =''


results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
                    coord_type, coord_grid, high_res=20., verbose=False, 
                    csv_output='', plot_output='')

# print(results['c3'])
# print( results['rho'])msisin_den_file = '/data/data_geodyn/results/st/msis2/msis2_acceloffSLR_kamodo_interface/DENSITY/' + "st031109_2wk.goco05s_msisin"

DEN1_csv = pd.read_csv(msisin_den_file, 
                    skiprows = 1, 
                    names = ['IYYDDD',
                             'IYR',
                              'DAY',
                             'UTSEC',
                             'ALTKM',
                             'GLAT',
                             'GLON',
                             'STLOC', 
                             'AVGFLX',
                             'FLUX',
                             'AP1',
                             'AP2',
                             'AP3',
                             'AP4',
                             'AP5',
                             'AP6',
                             'AP7',
                            ],
                    sep = '\s+',
                    )


DEN1_csv['Date'] = (pd.to_datetime('0'+ ((DEN1_csv['IYR'].astype(int).astype(str))),  format='%y') 
                    +  pd.to_timedelta(DEN1_csv['DAY'], unit='days'))



SWI_option = [1.0]*25
SWI_option[8] = -1.0


import matplotlib.pyplot as plt
import numpy as np
from pymsis import msis
import pandas as pd


msis2_df= pd.DataFrame(data={'Rho'   :np.ones(np.shape(DEN1_csv['Date']))*np.nan,
                            })

for index, row in DEN1_csv[:500].iterrows():

    lon = DEN_csv['Lon'][index] #row['GLON']
    lat = DEN_csv['Lat'][index] #row['GLAT']
    alts = DEN_csv['Height (meters)'][index]-subtract_alt
    f107 = DEN_csv['flux_daily'][index] # row['FLUX'] 
    f107a = DEN_csv['flux_avg'][index] # row['AVGFLX']

    aps = [[row['AP1'],row['AP2'],row['AP3'],row['AP4'],row['AP5'],row['AP6'],row['AP7']]]
    date = DEN_csv['Date'][index]  #row['Date']

    msis_data2 =  msis.run(date, lon, lat, alts, f107, f107a, aps, version=2,  options = SWI_option)
    msis_data2  = np.squeeze(msis_data2)


    msis2_df.loc[index, 'Rho'] = msis_data2[0]
    msis2_df.loc[index, 'Date'] = row['Date']


In [ ]:
results['c3']

In [ ]:

import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px


config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })

fig = make_subplots(rows=1, cols=1,
                   )

fig.add_trace(go.Scatter(y=results['rho'],
                         x=results['utc_time'],
                         name= 'Kamodo w/ Extrapolation',
                         mode='markers+lines',
                         ),
                         )
fig.add_trace(go.Scatter(x=DEN_csv['sattime_utctimestamp'][:500],
                         y=msis2_df['Rho'][:500]*1e-3,
                         name= 'MSIS2 direct read',
                         mode='markers',
                         marker=dict(size=3)),
                         )

fig.update_yaxes(type="log", exponentformat= 'power', row=1, col=1, title ='Density [g/cm^3]')
fig.update_xaxes(type="linear", exponentformat= 'power', row=1, col=1, title ='timestamp')
fig.update_layout(title="(LOWER ALTITUDE by "+str(subtract_alt)+"km) Comparing the Densities across the model options")
fig.show()


#===================================================================================================

import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px


config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })

fig = make_subplots(rows=1, cols=1,
                   )

fig.add_trace(go.Scatter(y=results['c3'],
                         x=results['rho'],
                         name= 'Kamodo w/ Extrapolation',
                         mode='markers+lines',
                         ),
                         )
fig.add_trace(go.Scatter(y=DEN_csv['Height (meters)'][:500]-subtract_alt,
                         x=msis2_df['Rho'][:500]*1e-3,
                         name= 'MSIS2 direct read',
                         mode='markers',
                         marker=dict(size=3)),
                         )

fig.update_yaxes(type="log", exponentformat= 'power', row=1, col=1, title ='Altitude')
fig.update_xaxes(type="linear", exponentformat= 'power', row=1, col=1, title ='Density')
fig.update_layout(title="(LOWER ALTITUDE by  "+str(subtract_alt)+"km) Comparing the Densities across the model options")
fig.show()

In [ ]:
results

In [ ]:
# import matplotlib.pyplot as plt
# plt.semilogy(results['c3'] ,results['rho']  ,'.', ms=6)
# # plt.semilogy(DEN_csv['sattime_utctimestamp'][:500] ,DEN_csv['rho (kg/m**3)'][:500],'.', ms=3)
# plt.semilogy(alts ,MANUAL_list,'.', ms=3)
# plt.legend(['Kamodo', 'manual'])

# plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(results['utc_time'] ,results['c1'] ,'.', ms=6)
plt.plot(DEN_csv['sattime_utctimestamp'][:500] ,DEN_csv['Lon'][:500],'.', ms=3)
plt.legend(['Kamodo', 'DEN_csv'])
plt.show()

import matplotlib.pyplot as plt
plt.plot(results['utc_time'] ,results['c2'],'.',ms=6 )
plt.plot(DEN_csv['sattime_utctimestamp'][:500] ,DEN_csv['Lat'][:500],'.',ms=3)
plt.legend(['Kamodo', 'DEN_csv'])
plt.show()


import matplotlib.pyplot as plt
plt.plot(results['utc_time'] ,results['c3'],'.',ms=6 )
plt.plot(DEN_csv['sattime_utctimestamp'][:500] ,DEN_csv['Height (meters)'][:500],'.',ms=3)
plt.legend(['Kamodo', 'DEN_csv'])
plt.show()

## The below proves that we need to get the full satellite flythrough working for TIEGCM

In [ ]:
# sys.exit()

In [ ]:
# %load_ext autoreload
# %autoreload 2
# from numpy import array
# import sys

# KAMODO_list = []


# for i,val in enumerate(DEN_csv['Elapsed Secs'][:5]) :  #calls_csv['Num'][:2]):
#     kamodo_program_path = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/SingleSatelliteFlythrough.py'
#     model     =  "4" #'4' #'TIEGCM'
#     file_dir  = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
#     var_list  = "['rho']"
#     dz_list   = "['0']"
#     coordtype = 'SPH'     
#     coordgrid = 'sph'    
#     high_res  = '20.'
#     sat_time1 = str(DEN_csv['YYMMDD'][i])  #"031115" #  
#     sat_time2 = str(DEN_csv['HHMMSS'][i])  #"120000" #1068897600        
#     utc_time  = str(datetime.timestamp(datetime.strptime(sat_time1+sat_time2, '%y%m%d%H%M%S').replace(tzinfo=timezone.utc)))
#     c1        = str(DEN_csv['Lon'][i]) #                         '100.0'         # lon
#     c2        = str(DEN_csv['Lat'][i]) #                         '32.5'          # lat
#     c3        = str(DEN_csv['Height (meters)'][i]) # '400'           # alt
# #     print(c3)
    
#     sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
#     from SingleSatelliteFlythrough import SingleModelFlythrough

#     ### Prepare input to the satellite Flythrough
#     if len(model)==1: 
#         model = int(model)
#     temp_str = var_list[1:-1].replace("'","").replace(' ','').replace('"','')
#     variable_list = temp_str.split(',')   #['rho','N_n']
#     if array([len(item) for item in variable_list]).max()==1:
#         variable_list = array(variable_list, dtype=int)
#     temp_str = dz_list[1:-1].replace("'","").replace(' ','').replace('"','')
#     dz = list(array(temp_str.split(','),dtype=int))  #[1,0], later treated as a boolean
#     sat_time = float(utc_time) #1426637500.0
#     c1 = float(c1) #x[R_E] or lon[deg]
#     c2 = float(c2) #y[R_E] or lat[deg]
#     c3 = float(c3) #z[R_E] or radius[R_E] or altitude[km]
#     coord_type = coordtype  #'SPH', 'GDZ', etc
#     if len(coord_type)==1: coord_type=int(coord_type)
#     coord_grid = coordgrid  #'car' or 'sph'
#     if len(coord_grid)==1: coord_type=int(coord_grid)
#     high_res = 20.      
 
#     results = SingleModelFlythrough(model, 
#                           file_dir, 
#                           variable_list,
#                           dz, 
#                           sat_time, 
#                           c1, 
#                           c2, 
#                           c3, 
#                           coordtype, 
#                           coordgrid, 
#                           high_res)
    
#     KAMODO_list.append(results['rho'])

    
# print('KAMODO_list',KAMODO_list)

In [ ]:
# %load_ext autoreload
# %autoreload 2

# from numpy import vectorize

# @vectorize
# def VecSingleModelFlythrough(model, file_dir, variable_list, dz, sat_time, c1, 
#                              c2, c3, coord_type, coord_grid, high_res):
    
#     results = SingleModelFlythrough(model, 
#                       file_dir, 
#                       variable_list,
#                       dz, 
#                       sat_time, 
#                       c1, 
#                       c2, 
#                       c3, 
#                       coordtype, 
#                       coordgrid, 
#                       high_res)

#     return(results)


In [ ]:
# %load_ext autoreload
# %autoreload 2




# results = VecSingleModelFlythrough(model*np.ones(np.size(list(DEN_csv['Lon'])),dtype=int  )  ,
#                          file_dir ,
#                          variable_list ,
#                          dz*np.ones(np.size(list(DEN_csv['Lon'])),dtype=int),
#                          sattime*np.ones(np.size(list(DEN_csv['Lon'])) ),
#                          list(DEN_csv['Lon']),
#                          list(DEN_csv['Lat']), 
#                          list(DEN_csv['Height (meters)']),
#                          coord_type, coord_grid, high_res
#                             )